In [1]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from copy import deepcopy
from furl import furl
import pandas as pd
import gzip
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Загрузка данных

In [2]:
with gzip.open('data/raw_vote.html.gz','rt') as f:
    soup = BeautifulSoup(f, from_encoding='cp-1251')

# Парсим дерево

In [3]:
root = soup.find("div", {"id": "jstree_demo_div"}).find('ul', recursive=True)

In [4]:
def get_children(root):
    es = []
    for r in root.parent.find('ul', recursive=False).findAll('li', recursive=False):
        for d in r.findAll('a', recursive=False, class_=lambda x: x != 'tree-open'):
            es.append(d)
    return es

In [5]:
result = []

def make_hierarchy(node):
    temp_obj = {}
    temp_obj['href'] = ''
    temp_obj['children'] = []
    
    try:
        children = get_children(node)
    except Exception as e:
        temp_obj['text'] = node.text.strip()
        temp_obj['href'] = node.attrs.get('href')
        temp_obj['path'] = node.tree_path
        result.append(temp_obj)
        return temp_obj
    
    for child in children:
        child.tree_path = deepcopy(node.tree_path)
        child.tree_path.append(child.text.strip())
        obj_child = make_hierarchy(child)
        
        temp_obj['children'].append(obj_child)

    return temp_obj

root.tree_path = []
hierarchy = make_hierarchy(root)

# Check
hierarchy['children'][0]['children'][0]['children'][0]['children'][0]['children'][0]

{'href': 'region/izbirkom?action=show&root=12000001&tvd=4014001103304&vrn=100100067795849&prver=0&pronetvd=null&region=1&sub_region=1&type=0&report_mode=null',
 'children': [],
 'text': 'УИК №1',
 'path': ['ЦИК России',
  'Республика Адыгея (Адыгея)',
  'Республика Адыгея (Адыгея) – Адыгейский',
  'Адыгейская',
  'УИК №1']}

# Приводим данные в нужный вид

In [6]:
for r in result:
    r['region'] = r['path'][1]
    r['oik'] = r['path'][2]
    r['tik'] = r['path'][3]
    r['uik'] = r['path'][4]
        
    r['url'] = urljoin('http://www.vybory.izbirkom.ru', r['href'])
    f = furl(r['url'])
    r['tik_code'] = f.query.params['root']
    r['uik_code'] = f.query.params['tvd']
    r['region_code'] = f.query.params['region']
    r['sub_region_code'] = f.query.params['sub_region']

In [7]:
df = pd.DataFrame.from_dict(result)
df = df[['region', 'region_code', 'oik', 'sub_region_code', 'tik', 'tik_code', 'uik', 'uik_code', 'url']]

# EDA

In [8]:
df.head()

,region,region_code,oik,sub_region_code,tik,tik_code,uik,uik_code,url
0,Республика Адыгея (Адыгея),1,Республика Адыгея (Адыгея) – Адыгейский,1,Адыгейская,12000001,УИК №1,4014001103304,http://www.vybory.izbirkom.ru/region/izbirkom?action=show&root=12000001&tvd=4014001103304&vrn=100100067795849&prver=0&pronetvd=null&region=1&sub_region=1&type=0&report_mode=null
1,Республика Адыгея (Адыгея),1,Республика Адыгея (Адыгея) – Адыгейский,1,Адыгейская,12000001,УИК №2,4014001103305,http://www.vybory.izbirkom.ru/region/izbirkom?action=show&root=12000001&tvd=4014001103305&vrn=100100067795849&prver=0&pronetvd=null&region=1&sub_region=1&type=0&report_mode=null
2,Республика Адыгея (Адыгея),1,Республика Адыгея (Адыгея) – Адыгейский,1,Адыгейская,12000001,УИК №3,4014001103306,http://www.vybory.izbirkom.ru/region/izbirkom?action=show&root=12000001&tvd=4014001103306&vrn=100100067795849&prver=0&pronetvd=null&region=1&sub_region=1&type=0&report_mode=null
3,Республика Адыгея (Адыгея),1,Республика Адыгея (Адыгея) – Адыгейский,1,Адыгейская,12000001,УИК №4,4014001103307,http://www.vybory.izbirkom.ru/region/izbirkom?action=show&root=12000001&tvd=4014001103307&vrn=100100067795849&prver=0&pronetvd=null&region=1&sub_region=1&type=0&report_mode=null
4,Республика Адыгея (Адыгея),1,Республика Адыгея (Адыгея) – Адыгейский,1,Адыгейская,12000001,УИК №5,4014001103308,http://www.vybory.izbirkom.ru/region/izbirkom?action=show&root=12000001&tvd=4014001103308&vrn=100100067795849&prver=0&pronetvd=null&region=1&sub_region=1&type=0&report_mode=null


##### Одно и то же, выкидываем

In [9]:
df[df['region_code'] != df['sub_region_code']].head()
df.drop('sub_region_code', axis=1, inplace=True)

In [10]:
df['region'].value_counts()

Московская область                          3625
город Москва                                3620
Республика Башкортостан                     3426
Республика Татарстан (Татарстан)            2824
Краснодарский край                          2750
Ростовская область                          2633
Свердловская область                        2538
Нижегородская область                       2466
Челябинская область                         2298
Красноярский край                           2196
Новосибирская область                       2031
город Санкт-Петербург                       2007
Иркутская область                           1956
Республика Дагестан                         1915
Саратовская область                         1890
Алтайский край                              1882
Пермский край                               1838
Омская область                              1808
Самарская область                           1776
Оренбургская область                        1766
Кемеровская область 

In [11]:
df['oik'].value_counts()

Республика Дагестан – Южный                                    1041
Курганская область – Курганский                                 992
Республика Бурятия – Бурятский                                  839
Омская область – Любинский                                      832
Республика Саха (Якутия) – Якутский                             811
Республика Мордовия – Мордовский                                804
Республика Башкортостан – Благовещенский                        771
Амурская область – Амурский                                     765
Орловская область – Орловский                                   736
Нижегородская область – Нижегородский                           734
Красноярский край – Енисейский                                  724
Белгородская область – Старооскольский                          697
Тверская область – Заволжский                                   680
Республика Башкортостан – Салаватский                           678
Республика Коми – Сыктывкарский                 

In [12]:
df['tik'].value_counts()

Октябрьская                                     475
Мурманская                                      285
Советская                                       251
Ленинская                                       250
Красноармейская                                 246
Каменская                                       227
Кировская                                       215
Первомайская                                    209
Павловская                                      204
Красногвардейская                               201
Нижний Новгород, Нижегородская                  198
Александровская                                 191
Белгородская городская                          189
Спасская                                        175
Красногорская                                   169
Алексеевская                                    152
Балашихинская                                   152
Нижний Новгород, Советская                      146
Тольятти, Автозаводская                         144
Территориаль

In [13]:
df['uik'].value_counts()

УИК №104     82
УИК №106     82
УИК №103     81
УИК №105     81
УИК №101     81
УИК №108     81
УИК №109     81
УИК №102     81
УИК №107     80
УИК №110     79
УИК №112     78
УИК №111     78
УИК №120     78
УИК №119     78
УИК №113     77
УИК №122     77
УИК №121     77
УИК №117     77
УИК №116     77
УИК №115     77
УИК №123     77
УИК №202     76
УИК №118     76
УИК №201     76
УИК №114     76
УИК №214     76
УИК №204     76
УИК №212     76
УИК №213     76
УИК №218     75
УИК №125     75
УИК №215     75
УИК №124     75
УИК №203     75
УИК №211     74
УИК №217     74
УИК №210     74
УИК №219     74
УИК №207     73
УИК №208     73
УИК №209     73
УИК №216     73
УИК №306     73
УИК №305     73
УИК №127     73
УИК №126     73
УИК №206     72
УИК №205     72
УИК №304     72
УИК №220     72
УИК №307     72
УИК №303     71
УИК №309     71
УИК №308     71
УИК №315     71
УИК №313     71
УИК №302     71
УИК №314     71
УИК №310     71
УИК №128     71
УИК №221     71
УИК №131     71
УИК №132

# Сохраняем

In [14]:
df.to_csv('data/metadata.csv.gz', compression='gzip', index_label='ind')
df2 = pd.read_csv('data/metadata.csv.gz', compression='gzip', index_col='ind')
all(df == df2)

True